# numpy, scipy and the van der Waals Equation of State saturation pressure
In this notebook we present numerical evaluation of density and residual properties for given values of pressure and temperature using the van der Waals Equation of State. Then, using these calculations we present numerical evaluation of the saturation pressure for liquid-vapor equilibra at given temperature.

> **Here you will meet the following topics:**
>- Thermodynamics: *equation of state*, *calculation of*: *density*, *residual properties*, *saturation point pressure*
>- Python: `numpy`:, `roots`; `scipy`: `optimize`

Supose you have to solve the following problem in a thermodynamics course:
>"Calculate the saturation pressure for pure hexane at the temperature of 220 K according to the van der Waals equation of state"

We will show how here to implement a function to calculate density of a pure fluid at given T and P.
Then, how to calculate the residual properties. And at last, how to calculate the saturation pressure.

Here we will use more resources of the package numpy

Therefore, import numpy:

In [1]:
import numpy as numpy

And we will also use the package scipy.

The following slightly different syntax for importing packages works similarly to the one already presented. Here, we import only a subset of the resources of scipy, the subset identified as optimize.

In [2]:
from scipy import optimize as optimize
#from <-- keyword "from"
     #scipy <-- name of the parent module
           #import <-- keyword "import"
                  #optimize <-- name of the desired subpackage
                           #as <-- keyword "as"
                              #optimize <-- alias name

Consider the *van der Waals* equation of state (Reid et al., 1987, see Eq 3-6.1 and Table 3-5, line 1)

$P=\frac{RT}{\bar{V}-b}-\frac{a}{{\bar{V}}^{2}}$

$a=\frac{27}{64}\frac{{{Tc}^{2}}{R^{2}}}{Pc}$


$b=\frac{{R}{Tc}}{{8}{Pc}}$

where $R$ is the gas constant 

$R = 8.3144598$ $\mathrm{{J} {mol^{−1}} {K^{−1}}}$

and critical properties for hexane are 

$Tc = 507.5$ $\mathrm{K}$


$Pc = 30.1$ $\times {10^6}$ $\mathrm{Pa}$

So lets code this information, as in notebook 2:

First the gas constant:

In [3]:
#gas constant
R = 8.3144598 #J.mol^−1.K^−1

Now the physical properties of the substance:

In [4]:
#pure component critical point of hexane
# Tc (K)
Tc = 507.5 #K

# Pc (bar)
Pc = 30.1*(10**6) #Pa

Then the EoS parameters:

In [5]:
a = 27/64*((Tc**2)*(R**2)/Pc)
print(a)

0.2495499363743344


In [6]:
b = (R*Tc)/(8*Pc)
print(b)

1.752320742732558e-05


Here, we are interested in calculating $\bar{V}$ for a given substance, therefore given $a$ and $B$, at some given condition of $P$ and $T$. For that purpose we convert the explicit $P(T,\bar{V})$ representation of the EoS to an implicit representation. We can then find a repreesentation that corresponds to a 3$^{\mathrm{rd}}$ degree polynomial in $\bar{V}$, with coefficients depending on $P$, $T$, $a$ and $b$. THerefore, for a given substance (given $a$ and $b$), at given $P$ and $T$, the density can be obtained by solving this polynomial for $\bar{V}$.

${P}{{\bar{V}}^3}−({P}{b}+{R}{T}){{\bar{V}}^2}+{a}{\bar{V}}−{a}{b}={0}$

Let's define the temperature and pressure at which we want to perform a density calculation.

In [7]:
#calculate density at what temperature and pressure?
T = 220. #K
P = 1*(10**5) #Pa

In [8]:
c3 = P #coefficient for v^3
c2 = -(P*b+R*T) #coefficient v^2
c1 = a  #coefficient v^1
c0 = -a*b  #coefficient v^0

In [9]:
v=numpy.roots([c3,c2,c1,c0]) #numpy .roots finds the roots of a polynomial
#coefficients must be provided in a list form, constructed using the square brackets
#result is the list of real and imaginary roots found.
print(v)

[  1.81721416e-02   1.16545544e-04   2.06475858e-05]


Here, after solving a 3$^{\mathrm{rd}}$ degree polynomial, we may obtain three real valued roots. If three roots are found in the interval $(b,\infty )$, the minimum  and maximum corresponds to liquid and vapor aggregation states, respectively, the intermediate root does not correspond to a phisycally relevant state.

According to the principle of minimum Gibbs energy, at given $P$, $T$ (and quantity of matter), the state with least Gibbs energy is the *stable equilibrium state*. Therefore we must compare the Gibbs energy of these *states* to reveal which corresponds to the *stable equilibrium*. While we are not able to calculate absolute values of Gibbs energy using an equation of state, we are able to calculate relative values of Gibbs energy for both vapor and liquid states with a common reference associated with the ideal gas state, the relative scale of gibbs energy we use here is the residual Gibbs energy.

In [10]:
[v_Liq, v_intermediate, v_Vap] = numpy.sort(v)
print(v_Liq)
print(v_intermediate)
print(v_Vap)

2.06475858431e-05
0.000116545543579
0.018172141638


## The residual Gibbs energy
For the origin and discussion of residual thermodynamics properties, see the references below. For the van der Waals EoS, the expression for the residual Gibbs energy, $G^{Res}$, is:

$${G^{Res}} = PV-RT-\frac{a}{\bar{V}} -RTln \left (\frac{P(\bar{V}-b)}{RT} \right )$$

> `Smith, van Ness, Abbott, 2005, 7th-pt-BR, Ch. (6.2, 6.3)
> O'Connell and Haile, 2005, Ch. 4
> Tester and Modell, 1997, Ch. 9.`

In [11]:
Residual_G = (P*v)-(R*T)-(a/v) - T*(R*numpy.log(P*(v-b)/(R*T))) 
print(Residual_G)

[  -11.92852652  5587.49740938  1954.82586937]


note now that the lowest gibbs energy yielded correspond to the liquid phase, that means at this pressure and temperature, the liquid phase is the stable equilibrium state.

## Calculation of saturation pressure

Now, the saturation pressure is the pressure $P$ that makes the difference in residual gibbs energy between trial liquid and vapor pahse equal zero. We cannot explicitate P as a function of the difference in residual gibbs energy in order
for that purpose we define of a function $\Delta G^{Res}$ of $P$
and we will solve it numerically, i.e. trying various values of $P$ according to a updating methodology and convergence criteria. To automate taht task we impkement a function **`def Delta_Residual_G(P):`**, which calculates the roots of volume, as before, and evaluates the residual gibbs energy in each of these states

In [12]:
#define equilibrium criteria: Residual_G_Liquid is equal to Residual_G_Vapor :. Delta_Residual_G = zero :. Delta_Residual_G < Tolerance
def Delta_Residual_G(P):
    c3 = P #coefficient for v^3
    c2 = -(P*b+R*T) #coefficient v^2
    c1 = a  #coefficient v^1
    c0 = -a*b  #coefficient v^0
    V = numpy.roots([c3,c2,c1,c0])
    Vl = min(V)
    Vv = max(V)
    #print(P, V) #if you uncomment this line you will see every iteration in the output section of this cell
    return (P*(Vl-Vv) + (-a)*(1/(Vl)-1/(Vv))+T*(R*numpy.log((Vv-b)/(Vl-b))))

what follows is similar to the work of Topliss et al.,:
> Topliss, Robert J., Dimitrios Dimitrelis, and John M. Prausnitz. 1988. “Computational Aspects of a Non-Cubic Equation of State for Phase-Equilibrium Calculations. Effect of Density-Dependent Mixing Rules.” Computers & Chemical Engineering 12 (5): 483–89. doi:10.1016/0098-1354(88)85067-1.

To calculate saturation pressure for a given T, first we isolate the range of pressure where fluid phase equilibria is feasible:
That is the range of pressure where there are 3 valid roots of volume, remember the isotherms presented in notebook 2. The maximum and minimum pressure that presents 3 valid roots for volume we will call `P_upper_plateau` and `P_lower_plateau` respectively. 

> given
> 
> $T$,
> 
> find all possible
> 
> $\bar{V}^*$,
> 
> so that
> 
> $\left(\frac{\partial P}{  \partial \bar{V}}\right)_{T}=0$
> 
> in
> 
> $\bar{V}^* \in (b,\infty)$.
> 
> Then, find the corresponding
> 
> $P^* = P(T,\bar{V}^*)$.

Analytical development of the partial derivative yields

$\left(\frac{\partial P}{  \partial \bar{V}}\right)_{T}=-\frac{RT}{(\bar{V}-b)^2} + \frac{2a}{\bar{V}^3}$

therefore in the stationary points

$-\frac{RT}{(\bar{V}-b)^2} + \frac{2a}{\bar{V}^3}=0$

which represents another 3rd order in V polynomial (not the same from the problem of finding V given P and T)

$-{RT}{\bar{V}^3} + {2a}{(\bar{V}-b)^2}=0$

$-{RT}{\bar{V}^3} + {2a}{(\bar{V})^2}+{2a}{(b)^2}+{2a}{(-2\bar{V}b)}=0$

In [13]:
#calculate each coeffcient of the polynomial-in-V representation of dP/dV=0
rho = R*T
mu = 2*(-a)
nu = - b*(a*(-4))
omega=- 2*(b**2)*(a*(1))
#and use the method "roots" from package "numpy" to calculate the roots of this polynomial and assing the solutions to the variable plateaus as an array
plateaus = numpy.roots([rho,mu,nu,omega])
print("plateaus:", plateaus)

#filter out values of plateu correspondign to non-physical volume (less than b or complex)
plateaus = plateaus[ numpy.where( plateaus > b ) ] #using the numpy function 'where

print("plateaus:", plateaus)
Vli = min(plateaus)
Vvi = max(plateaus)

#find the corrsponding values of pressure for each filtered volume
Psup = R*T/(Vvi-b)-a/(Vvi*Vvi)
Psupi=Psup*.999

Pinf = R*T/(Vli-b)-a/(Vli*Vli)

#note that the algorithm cannot work with pressure less than or equal to zero, therefore fix minimium pressure to slightly above zero if is resulted negative

if Pinf > 0:
    Pinfi = Pinf*1.001
else:
    Pinfi = 0.000000001

print("Vli:",Vli)
print("Vvi:",Vvi)
print("Psupi:",Psupi)
print("Pinfi:",Pinfi)

plateaus: [  2.33425728e-04   2.51661243e-05   1.42624090e-05]
plateaus: [  2.33425728e-04   2.51661243e-05]
Vli: 2.51661242908e-05
Vvi: 0.000233425727852
Psupi: 3888420.00342
Pinfi: 1e-09


now we must find the solution P
we ask package optmize to use method bissect to solve the equilibrium criteria just defined


In [22]:
P_sat = optimize.bisect(Delta_Residual_G, Pinfi, Psupi, xtol=1e-12, rtol=4.4408920985006262e-16, maxiter=100, full_output=True, disp=True)
print("Saturation pressure = [",str(psat[0]),"] bar")

Saturation pressure = [ 297561.1280524415 ] bar


in the first element (index 0) of the returning array of `optmize.bisect` we read the value of the saturation pressure
in the second element we read additional information about the execution of the numerical method

In [23]:
print(P_sat[1])

      converged: True
           flag: 'converged'
 function_calls: 50
     iterations: 48
           root: 297561.1280524415


## Conclusion


# External references
scipy, optimize [documentation](http://docs.scipy.org/doc/scipy/reference/optimize.html)

# Credits
* Initially developed in python 2.7 by Guilherme Carneiro Queiroz da Silva
* Adapted to ipynb with python 3.5 by Iuri Soter Viana Segtovich